In [1]:
#####################
# IMPORT LIBS
#####################

import lightgbm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from pathlib import Path
import wandb
import datetime
import os
import random
import joblib
import shutil

sns.set()

from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder
from scoring import local_scorer
import scipy

from lightgbm import LGBMClassifier, LGBMRegressor


#####################
# SET CONSTANTS
#####################

INPUT_PATH = Path('../input')
OUTPUT_PATH = Path('../output')
TRAIN_PATH = INPUT_PATH 

TARGET_COLUMNS = ['sale_flg', 'sale_amount', 'contacts']
FIXED_SEEDS = [948, 534, 432, 597, 103, 21, 2242, 17, 20, 29]

RANDOM_SEED = 4444
USE_WANDB = False
CURRENT_TIME = str(datetime.datetime.now()).replace(' ', '_').split('.')[0]

def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(RANDOM_SEED)

In [3]:
%%time

transactions = pd.read_csv(INPUT_PATH / 'trxn.csv')
assets_under_management = pd.read_csv(INPUT_PATH / 'aum.csv')
balance = pd.read_csv(INPUT_PATH / 'balance.csv')
client = pd.read_csv(INPUT_PATH / 'client.csv')
campaigns = pd.read_csv(INPUT_PATH / 'com.csv')
deals = pd.read_csv(INPUT_PATH / 'deals.csv')
dict_merchant_category_code = pd.read_csv(INPUT_PATH / 'dict_mcc.csv')
payments = pd.read_csv(INPUT_PATH / 'payments.csv')
funnel = pd.read_csv(INPUT_PATH / 'funnel.csv')
appl = pd.read_csv(INPUT_PATH / 'appl.csv')

/opt/conda/lib/python3.7/site-packages/decorator.py:231: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  return caller(func, *(extras + args), **kw)


CPU times: user 6.34 s, sys: 498 ms, total: 6.84 s
Wall time: 6.83 s


In [65]:
import googlemaps
from datetime import datetime

gmaps = googlemaps.Client(key='<YOUR-API-KEY-HERE>')

# Geocoding an address
geocode_result = gmaps.geocode('PENZA G Russia')

In [66]:
geocode_result

[{'address_components': [{'long_name': 'Penza',
    'short_name': 'Penza',
    'types': ['locality', 'political']},
   {'long_name': 'Gorod Penza',
    'short_name': 'Gorod Penza',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Penza Oblast',
    'short_name': 'Penza Oblast',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'Russia',
    'short_name': 'RU',
    'types': ['country', 'political']}],
  'formatted_address': 'Penza, Penza Oblast, Russia',
  'geometry': {'bounds': {'northeast': {'lat': 53.288824,
     'lng': 45.23116479999999},
    'southwest': {'lat': 53.101482, 'lng': 44.830076}},
   'location': {'lat': 53.2272903, 'lng': 45},
   'location_type': 'APPROXIMATE',
   'viewport': {'northeast': {'lat': 53.288824, 'lng': 45.23116479999999},
    'southwest': {'lat': 53.101482, 'lng': 44.830076}}},
  'place_id': 'ChIJ0fAKpGIAQUERAanLoexs7bo',
  'types': ['locality', 'political']}]

In [ ]:
address_mapping = {'name': [], 'corrected_name': [], 'lat': [], 'long': []}

In [45]:
from tqdm import tqdm_notebook

In [76]:
address_mapping = {'name': [], 'corrected_name': [], 'lat': [], 'long': []}

In [93]:
for i, address in tqdm_notebook(enumerate(transactions['txn_city'].unique())):
    if address != address:
        print(i, address)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


0it [00:00, ?it/s]

66 nan


In [83]:
for address in tqdm_notebook(transactions['txn_city'].unique()):
    if address != address:
        corrected_name = address
        lat = -1
        long = -1
    else:
        address_mapping['name'].append(address)
        geocode_result = gmaps.geocode(address + ' Russia')

        try:
            corrected_name = geocode_result[0]['address_components'][0]['long_name']
            lat = geocode_result[0]['geometry']['location']['lat']
            long = geocode_result[0]['geometry']['location']['lng']
        except:
            corrected_name = address
            lat = -1
            long = -1
    
    address_mapping['corrected_name'].append(corrected_name)
    address_mapping['lat'].append(lat)
    address_mapping['long'].append(long)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/13584 [00:00<?, ?it/s]

In [119]:
address_mapping['name'][68]

'KAZAN'

In [135]:
address_mapping['lat'][67]

55.7878944

In [132]:
new_address_mapping = (address_mapping).copy()

In [140]:
pd.DataFrame(new_address_mapping).to_csv('address_mapping.csv', index = None)

In [136]:
new_address_mapping['corrected_name'].insert(59, 'Yekaterinburg')
new_address_mapping['lat'].insert(59, 56.838011)
new_address_mapping['long'].insert(59,60.597465)

In [137]:
for i in range(70):
    print(i, new_address_mapping['name'][i], '->', new_address_mapping['corrected_name'][i])

0 N. NOVGOROD -> Nizhny Novgorod
1 RADUZHNYI -> Raduzhny
2 RADUZHNYJ -> Raduzhny
3 ORSK -> Orsk
4 KSTOVO -> Kstovo
5 SARATOV -> Saratov
6 PENZA -> Penza
7 PENZA G -> Penza
8 KAZAN -> Kazan
9 NOVOSIBIRSK -> Novosibirsk
10 PERM -> Perm
11 NOVOURALSK -> Novouralsk
12 TIHVIN -> Tikhvin
13 OREKHOVOZUEV -> Orekhovo-Zuyevo
14 KOGALYM G -> Kogalym
15 UFA -> Ufa
16 KOGALYM -> Kogalym
17 ROSTOVTNADO -> Rostov-on-Don
18 ROSTOVNADON -> Rostov-on-Don
19 TIKHVIN -> Tikhvin
20 NEFTEYUGANSK -> Nefteyugansk
21 CHEUSKINO -> Cheuskino
22 BERDYAUSH RP -> 12
23 SATKA -> Satka
24 VOLGOGRAD -> Volgograd
25 NEFTEYUANSK -> Nefteyugansk
26 DIMITROVGRAD -> Dimitrovgrad
27 RYAZANOVO -> Ryazanovo
28 SANKTPETERBURG -> Saint Petersburg
29 SANKTPETERBU -> Saint Petersburg
30 KOROLEV G -> Gorod Korolev
31 MOSKVA -> Moscow
32 PYTYAH -> Pyt-Yakh
33 KEMEROVO -> Kemerovo
34 VLADIKAVKAZ -> Vladikavkaz
35 BESLAN -> Beslan
36 VLADIKAVKAZ G -> Gorod Vladikavkaz
37 YUZHNOURALSK -> Yuzhnouralsk
38 CHELYABINSK -> Chelyabinsk
39 

In [127]:
address_mapping['name'][60]

'N. NOVGOROD'

In [128]:
address_mapping['corrected_name'][60]

'Raduzhny'